# Running Auto Correlation with Talon Deployer and BITE
###### Last Updated: 19/03/24

This demo will show the basic operation of auto correlation using the new Deployer and BITE TANGO devices currently deployed in this repository. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

## Pre-requisites


First, this notebook assumes you have a running environment launched from a pipeline, in particular it assumes you are running off one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes you are using a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry]().

### Running on Dev servers

Firstly, to make future commands a bit easier, we load the namespace name into a variable.

In [ ]:
NS = "ska-mid-cbf"


To run this locally, a few extra steps are required:
* Uncomment the [pv.yaml file in Engineering Console](https://gitlab.com/ska-telescope/ska-mid-cbf-engineering-console/-/blob/main/charts/ska-mid-cbf-engineering-console-umbrella/templates/pv.yaml). This will establish the persistant volume, without having to go through MCS.
* Run the required make commands in the CICD repo.
    * `make minkube-clean DRIVER=docker`- Clean out the old minikube if previously used.
    * `make all DRIVER=docker` - Set up the minkube using the docker driver.
    * `eval $(minikube docker-env)` - Set the docker enivronment vairable.
* Then run the make commands to set up in the EC repo.
    * `make oci-build-all` - Build latest images/requriments in the repo.
    * `make oci-image-build` - Check to make sure the latest image is built.
    * `make k8s-reinstall-chart` - Deploy these images to the minikube.

If you check the minkube pods, you should see the TANGO database pods, the itango console and the old bite and deployer pods running. If they are still starting, give them a few moments.

In [ ]:
!kubectl -n {NS} get pods

This should return something like: 
```
NAME                                         READY   STATUS    RESTARTS   AGE
databaseds-ds-tango-databaseds-test-0        1/1     Running   0          17m
databaseds-tangodb-tango-databaseds-test-0   1/1     Running   0          17m
ds-bite-bite-0                               1/1     Running   0          16m
ds-deployer-deployer-0                       1/1     Running   0          16m
ds-tangotest-test-0                          1/1     Running   0          16m
ska-tango-base-itango-console                1/1     Running   0          17m
```

### Running on PSI


To Run on PSI, once running the launch step from the PSI pipeline, grab your booted namespace's name. You can run a check on the ns as well to make sure the pods are ready.

In [ ]:
!kubectl get ns

To make future work a bit easier we can load this into a NS var.

In [ ]:
# Change this var to be the non-spd namespace with your username in it
NS = "ci-ska-mid-psi-1218359385-alexschell"

We can then check to ensure the pods have spun up successfully.

In [ ]:
!kubectl -n $NS get pods

## Initial Setup/Using Deployer Device

First we want to set up the target board that we will be using for the auto correlation. While this can be multiple boards, for now we will only need one. Punch out the board(s) you have access to and set the `TARGET_BOARD` to assign it to be used in future steps.

The `TANGO_HOST` will be created based of the namespace you set earlier.

In [ ]:
import os
# Set to the boards you wish to configure.
TARGET_BOARD = [3]


TANGO_HOST = "databaseds-tango-base."+NS+".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)

os.environ["TANGO_HOST"] = TANGO_HOST

For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [9]:
from PyTango import DeviceProxy, Database, exception

# Setup the device proxies targeting bite and deployer
db = Database()
deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")

#Check the devices initially deployed to the database
devices = db.get_device_exported("*")
print(devices)
# Make sure the deployer device is set to ON
deployer_tango.On()
print("State of deployer device:")
print(deployer_tango.state())

DbDatum[
        name = '*'
value_string = ['dserver/CbfController/controller', 'dserver/CbfSubarray/cbfsubarray-01', 'dserver/CbfSubarray/cbfsubarray-02', 'dserver/CbfSubarray/cbfsubarray-03', 'dserver/DataBaseds/2', 'dserver/ECBite/bite', 'dserver/ECDeployer/deployer', 'dserver/FspMulti/fsp-01', 'dserver/FspMulti/fsp-02', 'dserver/FspMulti/fsp-03', 'dserver/FspMulti/fsp-04', 'dserver/MidCspCapabilityFsp/capabilityfsp', 'dserver/MidCspCapabilityVcc/capabilityvcc', 'dserver/MidCspController/controller', 'dserver/MidCspSubarray/subarray1', 'dserver/MidCspSubarray/subarray2', 'dserver/MidCspSubarray/subarray3', 'dserver/PowerSwitch/powerswitch-001', 'dserver/PowerSwitch/powerswitch-002', 'dserver/PowerSwitch/powerswitch-003', 'dserver/PowerSwitch/powerswitch-004', 'dserver/Slim/mesh', 'dserver/SlimLink/fs-links', 'dserver/SlimLink/vis-links', 'dserver/TalonBoard/talon-001', 'dserver/TalonBoard/talon-002', 'dserver/TalonBoard/talon-003', 'dserver/TalonBoard/talon-004', 'dserver/TalonBoard

First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted.

In [10]:
deployer_tango.write_attribute("targetTalons",TARGET_BOARD)

With this set, we can then run the configuration command by calling the generate_config_jsons command.

In [11]:
deployer_tango.command_inout("generate_config_jsons")


We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

**! CURRENTLY THIS DOES NOT WORK ON MINIKUBE, MUST USE PSI !**

In [12]:
deployer_tango.command_inout("download_artifacts", timeout=None)

CommunicationFailed: DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_CallTimedout
  origin = Connection::command_inout()
  reason = API_CorbaException
severity = ERR]

DevError[
    desc = Timeout (3000 mS) exceeded on device mid_csp_cbf/ec/deployer, command download_artifacts
  origin = Connection::command_inout()
  reason = API_DeviceTimedOut
severity = ERR]
]

In [13]:
!kubectl logs -n $NS ds-deployer-deployer-0

1|2024-03-19T16:22:20.038Z|INFO|MainThread|write_loggingLevel|base_device.py#1085|tango-device:mid_csp_cbf/ec/deployer|Logging level set to LoggingLevel.INFO on Python and Tango loggers
1|2024-03-19T16:22:20.038Z|INFO|MainThread|update_logging_handlers|base_device.py#313|tango-device:mid_csp_cbf/ec/deployer|Logging targets set to ['tango::logger']
1|2024-03-19T16:22:20.145Z|INFO|MainThread|exit|core.py#136||Finished processing state UNINITIALISED exit callbacks.
1|2024-03-19T16:22:20.145Z|INFO|MainThread|enter|core.py#130||Finished processing state INIT enter callbacks.
1|2024-03-19T16:22:20.145Z|INFO|MainThread|_update_state|base_device.py#905|tango-device:mid_csp_cbf/ec/deployer|Device state changed from UNKNOWN to INIT
1|2024-03-19T16:22:20.146Z|INFO|MainThread|callbacks|core.py#1147||Executed callback '<bound method OperationStateMachine._state_changed of <ska_tango_base.state_machine.OperationStateMachine object at 0x7fc2ec0f6f80>>'
1|2024-03-19T16:22:20.147Z|INFO|MainThread|do|ba

Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command.

In [14]:
deployer_tango.command_inout("configure_db")

The TANGO database should now be configured with all the devices needed for the next step, running the BITE device.

In [15]:
print(*db.get_device_exported("*").value_string, sep="\n")

dserver/CbfController/controller
dserver/CbfSubarray/cbfsubarray-01
dserver/CbfSubarray/cbfsubarray-02
dserver/CbfSubarray/cbfsubarray-03
dserver/DataBaseds/2
dserver/ECBite/bite
dserver/ECDeployer/deployer
dserver/FspMulti/fsp-01
dserver/FspMulti/fsp-02
dserver/FspMulti/fsp-03
dserver/FspMulti/fsp-04
dserver/MidCspCapabilityFsp/capabilityfsp
dserver/MidCspCapabilityVcc/capabilityvcc
dserver/MidCspController/controller
dserver/MidCspSubarray/subarray1
dserver/MidCspSubarray/subarray2
dserver/MidCspSubarray/subarray3
dserver/PowerSwitch/powerswitch-001
dserver/PowerSwitch/powerswitch-002
dserver/PowerSwitch/powerswitch-003
dserver/PowerSwitch/powerswitch-004
dserver/Slim/mesh
dserver/SlimLink/fs-links
dserver/SlimLink/vis-links
dserver/TalonBoard/talon-001
dserver/TalonBoard/talon-002
dserver/TalonBoard/talon-003
dserver/TalonBoard/talon-004
dserver/TalonBoard/talon-005
dserver/TalonBoard/talon-006
dserver/TalonBoard/talon-007
dserver/TalonBoard/talon-008
dserver/TalonDxLogConsumer/001


## Running Commands Through the BITE Device

Now that the BITE tango Device has been deployed via the deployer, we can use it to configure tests. First we check the device is running.

In [16]:
bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite_tango.State())

RUNNING


Now that we have confirmed it is running, we load in what board we want to configure.

In [17]:
bite_tango.write_attribute("boards",TARGET_BOARD)

For now, we can use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [18]:
print(bite_tango.command_inout("generate_bite_data"))

{'talon': '3', 'dish_id': 'SKA063', 'sample_rate_k': 1220, 'bite_config_id': 'basic gaussian noise', 'bite_initial_timestamp_time_offset': 60.0}


Next, we actully start the LSTV replay.